<a href="https://colab.research.google.com/github/comojin1994/Deep_Learning_Study/blob/master/3step_lecture/Basic_setting_Expert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow-gpu==2.0.0

     |████████████████████████████████| 380.8MB 47kB/s 
     |████████████████████████████████| 450kB 43.7MB/s 
     |████████████████████████████████| 3.8MB 51.0MB/s 
ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 2.0.2 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 2.0.1 which is incompatible.
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0


In [2]:
import tensorflow as tf
print(tf.__version__)

2.0.0


### Set Hyperparameter

In [0]:
num_epochs = 5

### Define Model

In [0]:
class MyModel(tf.keras.Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.flatten = tf.keras.layers.Flatten(input_shape=(28, 28))
    self.d1 = tf.keras.layers.Dense(128, activation='sigmoid')
    self.d2 = tf.keras.layers.Dense(10, activation='softmax')

  def call(self, x):
    x = self.flatten(x)
    x = self.d1(x)
    x = self.d2(x)
    return x

### Define train step

In [0]:
@tf.function
def train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy):
  with tf.GradientTape() as tape:
    predictions = model(images)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)

  optimizer.apply_gradients(zip(gradients, model.trainable_variables))
  train_loss(loss)
  train_accuracy(labels, predictions)

### Define test step

In [0]:
@tf.function
def test_step(model, images, labels, loss_object, test_loss, test_accuracy):
  predictions = model(images)

  t_loss = loss_object(labels, predictions)
  test_loss(t_loss)
  test_accuracy(labels, predictions)

### Load dataset

In [0]:
mnist = tf.keras.datasets.mnist

(train_x, train_y), (test_x, test_y) = mnist.load_data()
train_x, test_x = train_x / 255., test_x / 255.

train_x = train_x[..., tf.newaxis]
test_x = test_x[..., tf.newaxis]

train_ds = tf.data.Dataset.from_tensor_slices((train_x, train_y)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((test_x, test_y)).batch(32)

### Create model

In [0]:
model = MyModel()

### Define loss_ftn, optimzier

In [0]:
loss_ftn = tf.keras.losses.sparse_categorical_crossentropy
optm = tf.keras.optimizers.Adam()

### Define metircs

In [0]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

### Training

In [26]:
for epoch in range(num_epochs):
  for images, labels in train_ds:
    train_step(model, images, labels, loss_ftn, optm, train_loss, train_accuracy)

  for test_images, test_labels in test_ds:
    test_step(model, test_images, test_labels, loss_ftn, test_loss, test_accuracy)

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print(template.format(epoch+1,
                        train_loss.result(),
                        train_accuracy.result() * 100,
                        test_loss.result(),
                        test_accuracy.result()*100))

Epoch 1, Loss: 0.13996483385562897, Accuracy: 95.92499542236328, Test Loss: 0.13382071256637573, Test Accuracy: 95.9800033569336
Epoch 2, Loss: 0.1238316223025322, Accuracy: 96.39583587646484, Test Loss: 0.1226944699883461, Test Accuracy: 96.2750015258789
Epoch 3, Loss: 0.11118458956480026, Accuracy: 96.76610565185547, Test Loss: 0.11327783763408661, Test Accuracy: 96.5133285522461
Epoch 4, Loss: 0.10088098794221878, Accuracy: 97.0754165649414, Test Loss: 0.1077711284160614, Test Accuracy: 96.66749572753906
Epoch 5, Loss: 0.09232432395219803, Accuracy: 97.33100128173828, Test Loss: 0.10239923000335693, Test Accuracy: 96.81999969482422


In [27]:
model.summary()

Model: "my_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  100480    
_________________________________________________________________
dense_1 (Dense)              multiple                  1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________
